In [4]:
import numbers
from datetime import *
class TimeZone :
    def __init__ (self , name , offset_hours , offset_minutes ) :
         
         if name is None or len(name.strip())==0:
              raise ValueError('Timezone name cannot be empty.')
         
         self._name=name.strip()

         if not isinstance(offset_hours,numbers.Integral) :
              raise ValueError('Hour offset must be an integer.')
         
         if not isinstance(offset_minutes,numbers.Integral) :
              raise ValueError('Minutes offset must be an integer.')
         
         if offset_hours < -59 and offset_hours > 59 :
              raise ValueError('Minutes offset must between -59 and 59 (inclusive).')
         
         offset=timedelta(hours=offset_hours , minutes=offset_minutes) 

         if offset < timedelta(hours= -12, minutes=0) and offset > timedelta(hours=14 , minutes=0) :
              raise ValueError('Offset must be between -12:00 and +14:00.')
         
         self._offset_hours=offset_hours
         self._offset_minutes=offset_minutes
         self._offset=offset

    def __eq__ (self , other ) :
              
              return isinstance(other , TimeZone)  and ( self._name == other._name and
                                                    self._offset_hours == other._offset_hours and
                                                    self._offset_minutes == other._offset_minutes)
         

    def __repr__(self) :
              
              return (f"TimeZone(name='{self._name}', "
                f"offset_hours={self._offset_hours}, "
                f"offset_minutes={self._offset_minutes})")
              

In [5]:
o1=TimeZone('MST', -7, 0)
o1

TimeZone(name='MST', offset_hours=-7, offset_minutes=0)

In [6]:
from datetime import *
from itertools import count
class Account :
    trancation_id=count(100)
    _interest_rate=0.5

    _tranction_code={
         'deposit': 'D',
        'withdraw': 'W',
        'interest': 'I',
        'rejected': 'X'
    }
    
    def __init__(self,account_no,name,sirname, timezone=None, initial_balance=0) :

        self._account_no=account_no
        self._first_name=name
        self._last_name=sirname
        self._balance=0

        if timezone==None :    
              timezone=TimeZone('UTC',0,0)

        self.timezone=timezone
        self._balance=float(initial_balance)


    
    @property
    def account_no(self):
         return self._account_no

    @property
    def first_name(self):
         return self._first_name
    
    @first_name.setter
    def first_name(self,value):
         self.validate_and_set_name('_first_name', value, 'First Name')

    @property
    def last_name(self):
         return self._last_name
    
    @last_name.setter
    def last_name(self,value) :
        self.validate_and_set_name('_first_name', value, 'First Name')

    @property
    def full_name(self):
         return self._first_name + self.last_name 

    @property
    def timezone(self) :
         return self._timezone

    @timezone.setter    
    def timezone(self, value) :
         if not isinstance(value , TimeZone ):
              raise ValueError('Time zone must be a valid TimeZone object.')
         self._timezone=value 
    
    @property
    def balance(self) :
         return self._balance
   
    @classmethod
    def get_interest_rate(cls):
      return cls._interest_rate

    
    @classmethod
    def set_interest_rate(cls , value) :
         
         if not isinstance(value , numbers.Real) :
              raise ValueError('Interest rate must be a real number')
         elif value <0 :
              raise ValueError('Interest rate cannot be negative.')
         
         cls._interest_rate=value
     

    def validate_and_set_name(self, property_name, value, field_title):
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f'{field_title} cannot be empty.')
        setattr(self, property_name, value)   
    
    @staticmethod
    def validate_real_number(value , min_value=None) :
         if not isinstance(value , numbers.Real ) :
              raise ValueError('Value must be a real number.')
         
         if value is not None and value < min_value :
                raise ValueError(f'Value must be at least {min_value}')
         
         return value
    
    def deposit(self , value) :
         value=Account.validate_real_number(value, min_value=0.01)
         transction_code=self._tranction_code['deposit']
         con_code=self.generate_confirmation_code(transction_code)
         self._balance+=value
         return con_code
    

    def withdraw(self , value) :
         value=Account.validate_real_number(value, min_value=0.01)
         
         accepted=False

         if self.balance - value < 0:
             transction_code=self._tranction_code['rejected']
         else :
             transction_code=self._tranction_code['withdraw'] 
             accepted=True
               
         con_code=self.generate_confirmation_code(transction_code)

         if accepted :
           self._balance-=value
  
         return con_code
    

    def pay_interest(self) :
         
         interest= self._balance *self.get_interest_rate()/100
         transction_code=self._tranction_code['interest'] 
         con_code=self.generate_confirmation_code(transction_code)
         return con_code
    

    
    def generate_confirmation_code(self,transction_code) :
      dt_str=datetime.now(timezone.utc)
      self.dt_str=dt_str.strftime('%Y%m%d%H%M%S')
      dt_as_tz=dt_str.astimezone(timezone(timedelta(hours=-7),'MST'))
      self.dt_as_tz=dt_as_tz.strftime('%Y%m%d%H%M%S')
      return f'{transction_code}-{self.account_no}-{self.dt_str}-{next(Account.trancation_id)}'  
 


In [7]:
a = Account('A100', 'Eric', 'Idle', timezone=TimeZone('MST', -7, 0), initial_balance=100)
print(a.balance)
print(a.deposit(150.02))
print(a.balance)
print(a.withdraw(0.02))
print(a.balance)
Account.set_interest_rate(1.0)
print(a.get_interest_rate())
print(a.pay_interest())
print(a.balance)
print(a.withdraw(1000))

100.0
D-A100-20230328095408-100
250.02
W-A100-20230328095408-101
250.0
1.0
I-A100-20230328095408-102
250.0
X-A100-20230328095408-103


In [9]:
a.first_name
a.first_name='chetan'

In [10]:
a.first_name

'chetan'